In [24]:
import itertools
import sqlparse
import re
from sqlparse.sql import IdentifierList, Identifier
from sqlparse.tokens import Keyword, DML

In [2]:
def is_subselect(parsed):
    if not parsed.is_group:
        return False
    for item in parsed.tokens:
        if item.ttype is DML and item.value.upper() == 'SELECT':
            return True
    return False

In [3]:
def extract_from_part(parsed):
    from_seen = False
    for item in parsed.tokens:
        if item.is_group:
            for x in extract_from_part(item):
                yield x
        if from_seen:
            if is_subselect(item):
                for x in extract_from_part(item):
                    yield x
            elif item.ttype is Keyword and item.value.upper() in ['ORDER', 'GROUP', 'BY', 'HAVING', 'GROUP BY']:
                from_seen = False
                StopIteration
            else:
                yield item
        if item.ttype is Keyword and item.value.upper() == 'FROM':
            from_seen = True


In [4]:
def extract_table_identifiers(token_stream):
    for item in token_stream:
        if isinstance(item, IdentifierList):
            for identifier in item.get_identifiers():
                value = identifier.value.replace('"', '').lower()                
                yield value
        elif isinstance(item, Identifier):
            value = item.value.replace('"', '').lower()
            yield value

In [5]:
def extract_tables(sql):
    # let's handle multiple statements in one sql string
    extracted_tables = []
    statements = list(sqlparse.parse(sql))
    for statement in statements:
        if statement.get_type() != 'UNKNOWN':
            stream = extract_from_part(statement)
            extracted_tables.append(set(list(extract_table_identifiers(stream))))
    return list(itertools.chain(*extracted_tables))

In [30]:
sql = """
SELECT a.time_updated_server/1000,
content,
nick,
name
FROM table1 a
JOIN table2 b ON a.sender_id = b.user_id
JOIN table3 c ON a.channel_id = c.channel_id
JOIN table4 d ON c.store_id = d.store_id
WHERE sender_id NOT IN
  (SELECT user_id
   FROM table5
   WHERE store_id IN ('agent_store:1',
                                     'ask:1'))
   AND to_timestamp(a.time_updated_server/1000)::date >= '2014-05-01'
   GROUP BY 1,2,3,4
   HAVING sum(1) > 500
   ORDER BY 1 ASC
    """

# remove the /* */ comments
sql = re.sub(r"/\*[^*]*\*+(?:[^*/][^*]*\*+)*/", "", sql)
# remove whole line -- and # comments
lines = [line for line in q.splitlines() if not re.match("^\s*(--|#)", line)]
# remove trailing -- and # comments
clean_query = " ".join([re.split("--|#", line)[0] for line in lines])
# split on blanks, parens and semicolons
tokens = re.split(r"[\s)(;]+", q)

tables = ', '.join(extract_tables(clean_query))
print('Tables: {0}'.format(tables))

Tables: test_table2, test_table1


In [20]:
ls=sqlparse.parse(sql)
type(ls)
for i in ls :
    print(i.tokens)
  

[<Newline ' ' at 0x19F770E0228>, <Whitespace ' ' at 0x19F77458E88>, <Whitespace ' ' at 0x19F77458EE8>, <Whitespace ' ' at 0x19F77458F48>, <DML 'SELECT' at 0x19F77458FA8>, <Whitespace ' ' at 0x19F77460048>, <Function 'count(...' at 0x19F77396A98>, <Whitespace ' ' at 0x19F77460228>, <Keyword 'from' at 0x19F77460288>, <Whitespace ' ' at 0x19F774602E8>, <Identifier 'test_t...' at 0x19F77396B88>, <Newline ' ' at 0x19F774603A8>, <Where 'WHERE ...' at 0x19F77396B10>]


In [86]:
def query_commentline_removal(sql_str):

    # remove the /* */ comments
    q = re.sub(r"/\*[^*]*\*+(?:[^*/][^*]*\*+)*/", "", sql_str)

    # remove whole line -- and # comments
    lines = [line for line in q.splitlines() if not re.match("^\s*(--|#)", line)]

    # remove trailing -- and # comments
    q = " ".join([re.split("--|#", line)[0] for line in lines])

    # scan the tokens. if we see a FROM or JOIN, we set the get_next
    # flag, and grab the next one (unless it's SELECT).
    
    # split on blanks, parens and semicolons
    tokens = re.split(r"[\s)(;]+", q)

    return tokens
    

In [87]:
def nooftablecount(tokens):

    result = set()    
    get_next = False   
     #No.of tables 
    for tok in tokens:       
        if get_next:
            if tok.lower() not in ["", "select"]:
                result.add(tok)            
            get_next = False
        get_next = tok.lower() in ["from", "join"]            
    return len(result)

In [88]:
def noofleftjoincount(tokens):

    result = set()    
    get_next = False   
     #No.of left join 
    for tok in tokens:      
        if get_next:
            if tok.lower() not in ["", "select"]:
                result.add(tok)            
            get_next = False
        get_next = tok.lower() in ["left join"]
            
    return len(result)

In [89]:
def noofrightjoinecount(tokens):
 
    result = set()    
    get_next = False   
     #No.of left join 
    for tok in tokens:       
        if get_next:
            if tok.lower() not in ["", "select"]:
                result.add(tok)            
            get_next = False
        get_next = tok.lower() in ["right join"]
            
    return len(result)

In [172]:
def noofinnerjoinecount(tokens):

    result = set()    
    get_next = False   
     #No.of left join 
    for tok in tokens:       
        if get_next:
            if tok.lower() not in ["", "select"]:
                result.add(tok)
            
            get_next = False
        get_next = tok.lower() in ["inner join","join"]
            
    return len(result)

In [173]:
def noofselectcolumns(tokens):
    result=set()
    get_next=True
    frompos=0
    for position, name in enumerate(tokens):
        if name.upper() =="SELECT" :
            selectpos=position+1   
        
        if name.upper() == "FROM":
            frompos=position
            break 
      
    for i in tokens[selectpos:frompos]:
        result.add(i)
    
    return len(result) 

In [178]:
def noofsubquerys(tokens):
    result=set()
    selectpos=[]
    
    for position, name in enumerate(tokens):
        if name.upper() =="SELECT" :
            result.add(position)   
    if  len(result)>1 :
        return len(result)-1
    else:
        return 0;
    

In [181]:
def noofgroupbycolumncount(tokens):

    result = set()    
    get_next = False   
     #No.of left join 
    for tok in tokens:       
        if get_next:
            if tok.lower() not in ["", "select"]:
                result.add(tok)
            
            get_next = False
        get_next = tok.lower() in ["group by"]
            
    return len(result)

In [182]:
token=query_commentline_removal(sql)
noofgroupbycolumncount(token)

0

In [180]:
token=query_commentline_removal(sql)
tablecount=nooftablecount(token)
leftjoincount=noofleftjoincount(token)
rightjoinecount=noofrightjoinecount(token)
innerjoinecount=noofinnerjoinecount(token)
selectcolumncount=noofselectcolumns(token)
noofsubquerycount=noofsubquerys(token)
print(tablecount)
print(leftjoincount)
print(rightjoinecount)
print(innerjoinecount)
print(selectcolumncount)
print(noofsubquerycount)


5
0
0
3
4
1
